In [ ]:
import os
import re
import gc
import string
import unicodedata
import operator
import numpy as np
import pandas as pd

from sklearn import utils
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import f1_score

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
# from keras import initializers, regularizers, constraints
from keras.layers import Activation
from keras.engine.topology import Layer
from keras.layers import (Input, Embedding, SpatialDropout1D, Bidirectional, Flatten, Dense) #CuDNNLSTM
from keras.initializers import glorot_normal, orthogonal
from keras.models import Model
from keras.callbacks import (EarlyStopping, ModelCheckpoint,
                             ReduceLROnPlateau)
import wordcloud

from tqdm import tqdm
tqdm.pandas()


In [ ]:
#from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

In [ ]:


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

In [ ]:
df_train = pd.read_csv(os.path.join('','df_train.csv'))

In [ ]:
df_test = pd.read_csv(os.path.join('','df_test.csv'))

In [ ]:
X_train = df_train['question_text']

In [ ]:
X_train = list(X_train)
X_train[0]

'Does provocative psychotherapy work ?'

In [ ]:
X_test = df_test['question_text']
X_test = list(X_test)
X_test[0]

'What is the best thing to do if your child experience bullying ?'

In [ ]:
y_train = df_train.target
y_train = list(y_train)

In [ ]:
y_test = df_test.target
y_test = list(y_test)

In [ ]:
import ktrain
from ktrain import text

In [ ]:
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=100, classes=[0,1])
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:388: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')



preprocessing train...
language: en
train sequence lengths:
	mean : 17
	95percentile : 38
	99percentile : 50


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 17
	95percentile : 39
	99percentile : 50


In [ ]:
learner.fit_onecycle(1e-5, 3)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/3
4000/4000 [==============================] - 1443s 361ms/step - loss: 0.2945 - accuracy: 0.8856 - val_loss: 0.2325 - val_accuracy: 0.9122
Epoch 2/3
4000/4000 [==============================] - 1446s 362ms/step - loss: 0.2190 - accuracy: 0.9184 - val_loss: 0.2153 - val_accuracy: 0.9183
Epoch 3/3
4000/4000 [==============================] - 1446s 362ms/step - loss: 0.1705 - accuracy: 0.9395 - val_loss: 0.2181 - val_accuracy: 0.9200


In [ ]:
learner.validate(class_names=[0,1])

              precision    recall  f1-score   support

           0       0.91      0.93      0.92     15918
           1       0.93      0.91      0.92     16082

    accuracy                           0.92     32000
   macro avg       0.92      0.92      0.92     32000
weighted avg       0.92      0.92      0.92     32000



array([[14747,  1171],
       [ 1388, 14694]])